# Data Wrangling with Python
Fall 2021

In this workshop, we'll dive deep into some techniques for cleaning and re-shaping data in Python using the [pandas](https://pandas.pydata.org/docs/) library. 

Here's what you can expect to practice:
 - working with CSV data from various sources
 - reshaping data for analysis
 - joining datasets on common elements
 - handling text and time series data
 - dealing with nulls and duplicates

## Research question

Calculate change in US home prices over a five-year period, using the Zillow Home Value Index, and compare with change in median household income. Identify regions where the two measures diverge.

## Data sources

 - [Zillow Home Value Index](https://www.zillow.com/research/data/): "A smoothed, seasonally adjusted measure of the typical home value and market changes across a given region and housing type. It reflects the typical value for homes in the 35th to 65th percentile range."
 - [US Census, American Community Survey](https://www.census.gov/programs-surveys/acs) data for median household income over the previous 12 months.

## Setting up

### Library imports

Let's import any libraries we'll need. 

We'll do most of our work in `pandas`, which should be available automatically in a Google Colab environment. 

In [ ]:
import pandas as pd

We'll also use a library that simplifies the process of retrieving Census data. We may need to install it first.

In [ ]:
from census import Census

### Links & API registration

At this time, you should also [register for an API key](https://api.census.gov/data/key_signup.html) so as to be able to retrieve datasets from census.gov. Once you have completed the form, you should receive your API key at the email address you provided. 

The following link will allows us to download Zillow Home Value Index (ZVHI) data. The data covers the time period between January 2000 and August 2021. Values are aggregated at the level of the zip code.

In [ ]:
zhvi_all_homes = 'https://files.zillowstatic.com/research/public_csvs/zhvi/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1631541070'

## Loading & exploring data

### Loading data from CSV

The pandas `read_csv` method can load data from a URL as well as a file, provided the data is in the proper format. 

In [ ]:
zhvi_df = pd.read_csv(zhvi_all_homes)

### Reshaping data

If we look at the DataFrame's columns, we can see that each month is represented by a discrete column heading, leading to a very wide table. This is done to compress the data to save storage space. (The names of the months appear only once, in the column headings.) 

In [ ]:
zhvi_df.columns

The table has 30K+ rows, one row per zip code.

In [ ]:
len(zhvi_df)

For analysis, it's often easier to work with data in so-called "long" format. Meaning that each row corresponds to a single observation. 

An observation is a specific measure of one or more variables. In this dataset, we have basically two variables: time and location (zip code). So how can we re-shape this data so that each row contains a single value for ZHVI? 

pandas has a handy method called `melt`, which is useful when you have several columns that all contain the same **kind** of measure. Here, each of the month columns contains the same measure, i.e., the ZHVI values for that month. So we can "melt" this table so that all the ZHVI values are in one column, and all the months in another. 

And what about the geographical columns? We will retain those, making them what pandas calls "ID variables" -- the idea being that the combination of those columns creates a set of unique identifiers. In this case, that's the geographical location identified by zip code. 

In the following method call, I'm referring to the geographical columns by position, which I can do since they precede all of the month columns, by slicing the DataFrame's `columns` object.

In [ ]:
z_df = zhvi_df.melt(id_vars=zhvi_df.columns[0:9], var_name='month', value_name='value')

Now our dataset is much bigger, though it has exactly the same data! But the new shape will make it easier to group our data and filter it in different way. 

In [ ]:
len(z_df)

### Correcting data types

It's useful to make sure the datatypes in your DataFrame make sense for what they represent. In this case, our zip codes should be strings, our values floats, and our months datetime objects.

The `dtypes` property displays the type of each column. In pandas, a type of `object` is either for a column of strings or of mixed data types.

In [ ]:
z_df.dtypes

#### Padding strings 

Let's convert the zip code fields to strings. One thing you might notice is that by treating the zip code (`RegionName`) as an integer, pandas has truncated zip codes that begin with zero. That could pose problems later, if we want to match this data against other data using the zip code. 

We can fix that by converting the `RegionName` column to a string and using the Python string method `zfill`, which adds zeroes to the beginning of a string to make it the required length. The argument to `zfill` is the **total number** of characters in the string. If the string to which you apple `zfill` contains fewer characters than the number you provide, the string will be padded with zeroes to fill it out to the required length. For example, `'7'.zfill(3)` returns `'007'`.

To use `zfill` on the `RegionName` column of our DataFrame, we first have to convert the data to strings, using the `astype` method. Then we can `apply` the `zfill` method to the data in that column. The pandas `apply` method, which takes as its argument another function, executes that function once for each value in the column. 

In [ ]:
z_df['RegionName'] = z_df['RegionName'].astype(str).apply(lambda x: x.zfill(5))

#### Converting dates and times

Now let's convert the `month` column to a `datetime` type. This makes it much easier to aggregate on time series. In this case, pandas can interpret the string correctly without our supplying a pattern. In other cases, it may be necessary to provide a second argument to `pd.to_datetime`, indicating the pattern of the string.

In [ ]:
z_df['month'] = pd.to_datetime(z_df['month'])

Now we can filter by parts of the date, for instance, by year. To do this, we use the special `dt` attribute, which has attributes corresponding to day, month, and year.

In [ ]:
z_df.loc[z_df.month.dt.year == 2021]

#### Customizing the numerical format

When dealing with dollar values in the millions, it is useful to limit the number of decimal places to 2 and to add commas separating the thousands, etc. We can set this option globally as follows.

Note that it does not affect the underlying representation of the data (which is still floating-point decimal), just the way it displays on screen.

In [ ]:
pd.set_option('display.float_format','{:,.2f}'.format)

In [ ]:
z_df

### Exploring the data

#### Null values

One thing we may want to know about dataset is how many rows have null values. There are other ways to accomplish this, but a straightforward way to get a total by column is to call `sum` on the result of the `DataFrame.isna` method. `isna` returns a new DataFrame where each cell is either `True` or `False`, depending on whether it's null or not. And the `sum` method simply "adds up" these Boolean values, treating `True` as 1 and `False` as 0. The result shows us how many null values are in each column.

In [ ]:
z_df.isna().sum()

We can see that the only columns with nulls are the `Metro` column and the `value` column. The `Metro` column refers to the zip code's metro area; rural zip codes would not have a metro area, so that makes sense. The nulls in the `value` column might be more problematic, depending on our analysis; these are instances where we don't have a valid observation. (Maybe no data were available.)

Here we also have an example of the principle that an aggregate operation on a DataFrame (`sum`) returns a pandas Series.

If we want to examine the null values, we can use the `isna` method to find them.

In [ ]:
z_df.loc[z_df.value.isna()]

#### Checking for duplicates

Another problem frequently encountered is duplicated data. We can check for that using the `duplicated` method, which works like `isna` except that it returns `True` if a value is duplicated within a column. By default, it doesn't mark the first duplicate as a duplicate, but I usually find it helpful to see all the duplicates, so I pass the parameter `keep=False`, which means, "Count the first instance of a duplicated datum as a duplicate."

In [ ]:
zhvi_df.loc[zhvi_df.RegionName.duplicated(keep=False)]

No zip codes in our original dataset are duplicated, so that's good! Note that I didn't call that method on our "melted" dataset, since converting the original table to a "long" version created many duplicates. 

In our "melted" dataset, the combination of zip code and month should be unique. We can check for this using the `subset` parameter.

In [ ]:
z_df.loc[z_df.duplicated(subset=['RegionName', 'month'], keep=False)]

### Getting ACS data

To make things more interesting, we're going to combine our Zillow data with a dataset from the US Census. The American Community Survey provides five-year estimates of many important economic indicators, including median household income by zip code. 

We can use the Census API to fetch the tables we need. The Python [census](https://pypi.org/project/census/) package provides a convenient wrapper around the API's. 

If you submitted the form at the start of our workshop, you should have received an API key via the email address you provided. If you didn't, I'll provide a link to the datasets we're going to use. 

#### Retrieving Census data 

First we need to initialize the module with our API key.

In [ ]:
apikey = ''
census = Census(apikey)

To access Census tables by API, we need to know the specific variable names. You can find these in the [ACS documentation](https://api.census.gov/data/2019/acs/acs5/variables.html). Using the `census` module, we can specify the ACS 5-year tables and zip code as the geographical level. In addition, you can specify a year as a method parameter, if we want data other than that from the most recent survey. 

In order to have some timeseries data, we'll retrieve the data for 2014 and 2019. (It's recommended to use [non-overlapping datasets](https://www.census.gov/data/developers/data-sets/acs-5year.html) for the ACS 5-year surveys because of how the estimates are calculated.)

In [ ]:
median_income = "B19013_001E"
census = Census(apikey)
acs2019 = census.acs5.zipcode(median_income, '*')
acs2014 = census.acs5.zipcode(median_income, '*', year=2014)

#### Creating a DataFrame

The Census API returns the table as a list of dictionaries. You can see that the value for our variable -- median household income -- is provided for each row, as well as a code for the state and the zip code (both strings). 

Our first step is to convert these two lists to DataFrames so that we can efficiently merge them with our Zillow data.

In [ ]:
acs2019

pandas has a handy method call `DataFrame.from_records`, which creates a DataFrame from exactly this structure. (The elements of the list are the rows, the keys of the dictionaries are the columns.)

We can also rename our columns for clarity and concision.

In [ ]:
acs_df = pd.DataFrame.from_records(acs2019)
acs_df = acs_df.rename(columns={'B19013_001E': '2019_median_hhi',
                               'zip code tabulation area': 'zip_code'})

In [ ]:
len(acs_df)

#### Concatenating DataFrames

But we have two separate datasets, one from 2019 and one from 2014. Ideally, we want a single table containing all of our ACS data. We'll accomplish that by using pandas' `concat` method, which takes a list of DataFrames and stacks them one on top of the other.

We'll add a column to record the year. And we'll put this code into a Python function, which is good practice for encapsulating our code for reproducibility.

In [ ]:
def create_acs_df(datasets, years):
    '''
    Accepts a list of pandas DataFrames to concat and a list of years, which will be added as a column to the resulting DataFrame
    '''
    # Step 1: Create an empty DataFrame -- for the first dataset, we need something to concat it with
    df_all = pd.DataFrame()
    # Step 2: Create a for loop: we can use the zip method to loop over the datasets and years at the same time
    for dataset, year in zip(datasets, years):
        # Create a DataFrame from the current dataset
        df = pd.DataFrame.from_records(dataset)
        # Create a year column and populate it with the corresponding year
        df['year'] = year
        # Rename the columns
        df = df.rename(columns={'B19013_001E': 'median_hhi',
                               'zip code tabulation area': 'zip_code'})
        # Concat with the previous DataFrame
        df_all = pd.concat([df_all, df])
    # Don't forget to return something!
    return df_all

In [ ]:
acs_df = create_acs_df([acs2019, acs2014], [2019, 2014])

In [ ]:
acs_df

#### Dealing with "bad" data

It looks like we have some outliers here, which may be an artifact of how the data has been reported/coded. We can use the `describe` method to try to spot these. (Ignore the year column; we're only interested in outliers in the `median_hhi` column.

In [ ]:
acs_df.describe()

It doesn't really make sense to have negative median income; and indeed, by looking at the unique values in the column that fall **below** zero, we can see that there's only one value, which may mean any number of things, but for the purposes of this example, we can just discard those rows.

In [ ]:
acs_df.loc[acs_df.median_hhi < 0].median_hhi.unique()

We're using `DataFrame.loc` to limit to the rows with a non-negative value for `median_hhi`. We'll make a copy of the filtered DataFrame, which helps avoid issues as we manipulate this data later.

In [ ]:
acs_df = acs_df.loc[acs_df.median_hhi >= 0].copy()

In [ ]:
acs_df.describe()

### Building our joint dataset

In order to compare Zillow home values and median household income by geographical area, it will be useful to have a single table that contains both variables. We'll create this table in a few steps.

#### Creating aligned data

Most important is to make sure that our two variables are aligned, meaning that they represent observations at the same scale. 

How well are our datasets aligned?

| Dataset | Measure | Geographic Dimension | Temporal Dimension |
| --- | --- | --- | --- |
| Zillow | Estimated median home value (dollars) | Zipcode | Month |
| ACS | Median household income (dollars) | Zipcode | Year |

Zillow and ACS are measuring different things, but they're both using the median as their statistic, and they're both measuring value in dollars. Geographically, both datasets are aligned at the level of zipcode. 

However, the Zillow data are presented on a monthly basis, while the ACS data are available only at the annual level. Moreover, because of the 5-year overlap, we have only two "years" of ACS data to work with. We'll need to adjust our Zillow dataset to bring it into alignment with our ACS data.


(Technically, the ACS data represent five-year aggregates, which are not the same as annual measures. In what follows we are going to take a statistically naive approach to comparing and summarizing these data. Tis approach is not intended to be empirically rigorous; rather, it is meant to illustrate operations with pandas. For more on the appropriate methodology for handling ACS median values, [this guide](http://www.dof.ca.gov/Forecasting/Demographics/Census_Data_Center_Network/documents/How_to_Recalculate_a_Median.pdf) is a good starting point.)

#### Filtering a dataset and checking for missing data

First we limit the Zillow data to the years for which we have ACS data: 2014 and 2019. We can use the `dt` datetime attribute on the `month` column to select particular years with `DataFrame.loc`.

In [ ]:
z_df = z_df.loc[(z_df.month.dt.year == 2019) | (z_df.month.dt.year == 2014)].copy()

To see if there are zipcodes represented in each dataset not present in the other, we can use the `asin` method to compare the values in one DataFrame with the values in another. This code counts how many unique zip codes are in our Zillow dataset that are not present in the ACS dataset

In [ ]:
len(z_df.loc[~z_df.RegionName.isin(acs_df.zip_code)].RegionName.unique())

We can do the same to count how many zip codes in the ACS dataset are missing from the Zillow data.

In [ ]:
len(acs_df.loc[~acs_df.zip_code.isin(z_df.RegionName)])

In this case, we'll simply omit these non-overlapping data points from our merged dataset. But in other cases, it might be important to account for them in some way.

#### Aggregating by time scale

We'll aggregate the Zillow data by year in order to compare with them the ACS data at the same time scale. 

We'll use the `DataFrame.groupby` method to group our dataset at the desired level. We're grouping by year, so we can use the `dt.year` attribute on the `month` column as the group key. We can group by multiple columns, so we might want to include other columns, too (except for the `value` column, which contains the data that we're trying to summarize).

We can provide them as a list to the `groupby` method, which use the unique combinations of the values in these columns as the keys for grouping.

In [ ]:
z_grp = z_df.groupby([z_df.StateName, z_df.City, z_df.Metro, z_df.RegionName, z_df.month.dt.year])

Now let's take the `mean` of the `value` column from our grouped DataFrame. Such operations by default return a Series (in this case, with a so-called hierarchical index). But to turn that back to a DataFrame, we can use the `reset_index` method.

In [ ]:
z_means = z_grp.value.mean()
z_means = z_means.reset_index()

In [ ]:
z_means

When grouping by multiple columns, it's important to be careful when including columns with null values. Recall from our analysis above that sometimes the `Metro` column is blank in this dataset. 

What happens to those rows in our final group?

In [ ]:
z_means.loc[z_means.Metro.isnull()]

By grouping by a column with nulls, we've actually lost a fair amount of data! 

In [ ]:
len(z_df.loc[~z_df.RegionName.isin(z_means.RegionName)].RegionName.unique())

To fix this problem, we could omit the `Metro` column from our `groupby` statement. In more recent versions of pandas, we can also add a `dropna=False` parameter to our `groupby` statement, which will keep the null values in the group keys.

In [ ]:
z_grp = z_df.groupby([z_df.StateName, z_df.City, z_df.Metro, z_df.RegionName, z_df.month.dt.year],
                    dropna=False)
z_means = z_grp.value.mean()
z_means = z_means.reset_index()
z_means.loc[z_means.Metro.isnull()]

In [ ]:
z_means

#### Merging data

Now we're ready to merge our Zillow data with our ACS data. The `merge` method performs the equivalent of a SQL join on two DataFrames that share at least one common key. The keys are the columns with values that are the same in both datasets. The following chart shows the keys between our two datasets.

| Measure or Dimension | `z_means` | `acs` |
| --- | --- | --- | 
| State | `StateName` |  | 
| City | `City` |  |
| Metro | `Metro` |  |
| Zip code | `RegionName` | `zip_code`  |
| Year | `month` | `year` |
| State code |  | `state` |
| Median income | | `median_hhi` |
| Home value | `value` |  |

The keys don't have to have the same column **names**, but they must share at least some of the same **values** in those columns. In our case, the shared values are the zip codes and the years. (Note that both our Zillow and ACS datasets have columns for U.S. state, but the values in those columns do not overlap: the Zillow dataset uses the name of the state, while the ACS data uses a numerical identifier.)

Before merging our datasets, we'll rename the Zillow columns `month` and `value` to more accurately represent their contents. This isn't necessary for merging, but it will make the merged table more legible for us.

In [ ]:
z_means = z_means.rename(columns={'month': 'year',
                                 'value': 'zhvi'})


Now we do the merge, which creates a new DataFrame. The arguments to the `merge` method are as follows:

- `acs_df`: the second DataFrame we want to merge with the first.
- `left_on`: this parameter takes a list of columns in the first DataFrame to use as keys.
- `right_on`: this parameter takes a list of columns in the second DataFrame to use as keys.

You can also specify a `how` parameter, which indicates the kind of join. The default is an **inner** join, meaning that the merged DataFrame will only contains rows where the keys are present in both of the source DataFrames. In this case, an inner join will drop those rows for zip codes that are missing from either the Zillow or the ACS data.

In [ ]:
z_merged = z_means.merge(acs_df, left_on=['RegionName', 'year'], 
                         right_on=['zip_code', 'year'])

We don't need the `state` column from our ACS dataset. Nor do we need two columns of zip codes, so we can drop these from our merged dataset. The `axis` parameter is important in the `drop` method: `axis=1` means "drop columns" (as opposed to rows).

In [ ]:
z_merged = z_merged.drop(['state', 'zip_code'], axis=1)

#### Calculating change over time

As an illustration of further uses for `groupby`, we'll calculate the percentage change between 2014 and 2019 for both the ZHVI and median household income for each zip code.

First, we'll sort the merged dataset by year.

In [ ]:
z_merged = z_merged.sort_values(by='year')

Now we can create new columns to hold the percentage change for each of our measures (home value and income). To calculate the percentage change, we will use the built-in `pct_change` method, applying it to the result of a `groupby` operation (since we still want to observe the changes at the zip-code level).

In [ ]:
z_merged.columns

In [ ]:
columns = ['StateName', 'City', 'Metro', 'RegionName']
z_merged['zhvi_pc'] = z_merged.groupby(columns, dropna=False).zhvi.pct_change()
z_merged['hhi_pc'] = z_merged.groupby(columns, dropna=False).median_hhi.pct_change()

Note that the `pc` columns have null values for all the rows where the `year` is 2014. That is expected: the first value in each sequence represents no change (since it's the first value). For each subsequent value, the percentage change is calculated with respect to the preceding value.

In [ ]:
z_merged

If we're just interested in the percentage change, we could drop the 2014 rows.

In [ ]:
z_merged = z_merged.loc[z_merged.year == 2019].copy()

In [ ]:
z_merged.describe()

And here's a demonstration of how we could use a custom function to calculate percentage change. There's no need to do so -- the built-in method will be more efficient. But sometimes you want to compute an aggregation that's not available as a built-in method.

For relatively simple calculations, we can use a `lambda` function, which is basically a one-line Python function. We define it with `lambda` instead of `def`. 

The `x` in the lambda expression is a function parameter. When used with `apply` on the `zhvi` column, `x` will represent a pandas Series containing the ZHVI values for each group created by our `groupby` expression. 

Since `x` is a Series, it has all the usual Series methods and properties, including `iloc`, which lets us take the first and second values for the purposes of calculating the percentage change.

In [ ]:
z_means.groupby(['RegionName', 'StateName', 'City']).zhvi.apply(lambda x: (x.iloc[1] - x.iloc[0]) / x.iloc[0])

We can now use our merged, aggregated dataset to investigate certain questions.

For example, where have home values risen *faster* than household income?

In [ ]:
z_merged.loc[z_merged.zhvi_pc > z_merged.hhi_pc]

Where have they kept pace or lagged behind income?

In [ ]:
z_merged.loc[z_merged.zhvi_pc <= z_merged.hhi_pc]

### Questions for practice

1. Which states have the greatest range in home values? Hint: select year, group by state, calculate max - min


2. Can you find any cities where the *average* percent change in median income is higher than the *average* percent change in home values?


3. Are the most expensive cities (to buy in) the wealthiest (by median income)?
-------------


1. Answer

   a. We need to pick a point in time: let's say 2019. 

In [ ]:
z19 = z_means.loc[z_means.year == 2019]

1. b. Then we can group by state and use `apply` with a lambda function to find the difference between the largest and smallest values in each group. 

In [ ]:
z_19_range = z19.groupby('StateName').zhvi.apply(lambda x: x.max() - x.min())

1. c. Finally, we can sort the values in descending order to find the biggest differences.

In [ ]:
z_19_range.sort_values(ascending=False)

2. Answer

   a. We can group our merged dataset by city and state.

In [ ]:
city_pc = z_merged.groupby(['City', 'StateName'])

2. b. To compute the mean of two columns at the same time, we can use the `agg` method available on a pandas `GroupBy` object. This method takes a dictionary mapping column names to the name of a built-in pandas method or a lambda function.

In [ ]:
city_pc = city_pc.agg({'zhvi_pc': 'mean',
                             'hhi_pc': 'mean'})

2. c. Now we can use `.loc` to find those rows where our mean ZHVI percentage change is smaller.

In [ ]:
city_pc.loc[city_pc.zhvi_pc < city_pc.hhi_pc]

3. Answer
  
  a. First we group by state and city (to account for cities that may have the same names in different states) and take the mean of the ZHVI. We sort so that the highest numbers are at the top.

In [ ]:
zhvi_cities = z_merged.groupby(['StateName', 'City']).zhvi.mean().sort_values(ascending=False)

3. b. We do the same for median income.

In [ ]:
hhi_cities = z_merged.groupby(['StateName', 'City']).median_hhi.mean().sort_values(ascending=False)

3. c. For each of these, we take the top 100 rows (after resetting the index).

In [ ]:
top_zhvi_cities = zhvi_cities.reset_index().head(100)
top_hhi_cities = hhi_cities.reset_index().head(100)

3. d. Now we can use `merge` to find those cities in the top 100 for ZHVI that are also in the top 100 for median income. Since the `StateName` and `City` columns are common to both of the DataFrames we're mering we don't have to specify them as the keys for merging -- pandas will use them by default.

In [ ]:
top_zhvi_cities.merge(top_hhi_cities)